<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.sofisa.com.br/wp-content/uploads/2018/05/new-sofisa.com_.br_.svg" alt="Sofisa" width="220" align="right" />

# FIAP Challenge Sofisa - Demonstração do Teradata Vantage
    
</header>


Cesar Augusto da Silva - educador.cesar@gmail.com

Lucas Alexandre Alves da Silva - lucas.alexandre1@hotmail.com

Marivaldo dos Santos Barbosa - marivaldo.barbosa@icloud.com

Thiago Alberto de Souza Colen - thiago.souzacolen@gmail.com



Como protótipo da utilização da arquitetura escolhida pela equipe, vamos demonstrar neste Jupyter Notebook um exemplo de utilização do Teradata Vantage facilitando o dia-a-dia do Cientista de Dados por meio de "levar o analítico ao dado" ao invés de "levar o dado ao analitico".
Utilizaremos o Teradata SQL Kernel, disponível gratuitamente através do downloads.teradata.com. Com esse Kernel instalado, é possível realizar consultas no Teradata utilizando o mesmo JupyterHub que os usuários (Cientistas de Dados) já costumam a utilizar.


Neste exemplo, utilizaremos dados estruturados dentro do banco de dados Teradata Vantage, onde a partir do mesmo, habilitamos o Cientista de Dados à executar modelos de predição simples e que o auxiliará na criação de variáveis, de maneira eficaz e rápida, por não precisar levar o dado à outra plataforma.


Apresentamos então dois modelos de regressão, muito utilizados no setor financeiro, Regressão Linear e Regressão Logística.


### Regressão Linear

A Regressão Linear é um dos tipos fundamentais de algoritmos de modelagem preditiva. Na regressão linear, uma variável numérica dependente é expressa em termos da soma de uma ou mais variáveis numéricas independentes, cada uma multiplicada por um coeficiente numérico, geralmente com um termo constante adicionado à soma das variáveis independentes. A regressão linear consiste nos coeficientes das variáveis independentes juntamente com um termo constante que compõe um modelo de regressão linear. A aplicação desses coeficientes às variáveis (colunas) de cada observação (linha) em um conjunto de dados (tabela) é conhecida como "Scoring".

### Regressão Logística

A Regressão Logística é um dos tipos de análise estatística mais amplamente utilizados. Na Regressão Logística, um conjunto de variáveis independentes (neste caso colunas) é processado para prever o valor de uma variável dependente (coluna) que assume dois valores referidos como resposta (1) e não-resposta (0). Na verdade, o usuário especifica qual valor da variável dependente ele irá tratar como a resposta, e todos os outros valores assumidos pela variável dependente são tratados como não-resposta. O resultado não é, entretanto, uma variável numérica contínua como visto na Regressão Linear, mas sim uma probabilidade entre 0 e 1 de que o valor de resposta é assumido pela variável dependente.


---

1) Primeramente, vamos realizar o setup da variavel SystemName indicando a qual banco de dados Teradata iremos nos conectar.

In [1]:
%var SystemName=TRDT01

2) Em seguida, através da variável QLID, vamos identificar em qual database/usuário iremos armazenar os resultados de nossas consultas e predições. 

In [2]:
%var QLID=NOS_USR

3) A variável a seguir indica onde estão localizadas as bibliotecas utilizadas para nossa modelagem. Nesse caso, para facilitar a demonstração, utilizamos o nosso mesmo usuário acima.

In [3]:
%var XSPDB=NOS_USR

---

Através do comando %connect abaixo, iremos nos conectar ao banco de dados definido anteriormente.

In [4]:
%connect ${SystemName}

User Name: nos_usr
Password: ·······


Success: 'TRDT01' connection established and activated for user 'nos_usr'


Vamos definir o database default, de utilização, para onde estão localizadas as bibliotecas para modelagem.

In [5]:
DATABASE ${XSPDB};

Success: 1 rows affected

### DataSet de Demonstração - Financeiro -  Customers/Accounts/Transactions

Os dataset utilizado foi armazenado no nosso database de trabalho. É um dataset bancário fictício com a seguinte volumetria: tabela de clientes (customer) com ~10 mil linhas, tabela de contas (accounts) com ~100 mil linhas e tabela de transações (transactions) com ~1 milhão de linhas. Tais tabelas estão relacionadas seguindo o diagrama abaixo:


<a href="https://ibb.co/zQR4dTt"><img src="https://i.ibb.co/xXj7Ztr/DemoData.png" alt="DemoData" border="0" /></a>

In [6]:
SELECT * FROM ${XSPDB}.Customer SAMPLE 10;

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
1,1363068,57612,58,8,2,M,2,Mr.,Jester,Pauly,11583,Rose,60606,Chicago,IL
2,1362500,56708,40,0,0,M,1,,Dan,Young,15242,Clairemont Mesa,75370,Dallas,TX
3,1363199,0,16,1,0,M,1,,Alfred,Maloney,18391,J,94225,Sacramento,CA
4,1363088,28975,54,4,2,M,2,,Johnson,Johnston,7625,35th,78738,Austin,TX
5,1363213,38010,60,0,0,M,2,Mr.,Jaime,Keith,5268,Lemon,44127,Cleveland,OH
6,1362720,18806,19,4,0,M,1,,Dyan,Schimmelpfennig,5042,C,60665,Chicago,IL
7,1362805,4626,78,8,0,F,3,Ms.,Dana,Williamson,6006,44th,10105,New York City,NY
8,1362941,18377,36,6,2,F,2,,Alice,Russell,13946,Leek,77089,Houston,TX
9,1362920,5781,80,0,0,F,1,,Claudia,Harrison,201,Oak,15263,Pittsburgh,PA
10,1363247,19686,64,4,0,F,2,Ms.,Michelle,Raphael,15108,32nd,43653,Toledo,OH


In [7]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2021.04.02_21.36.10.568_UTC
History ID:    936
Rows:          10 of 10
Parts:         2
Column Definitions:
    cust_id: INTEGER
    income: INTEGER
    age: SMALLINT
    years_with_bank: SMALLINT
    nbr_children: SMALLINT
    gender: CHAR(1)
    marital_status: CHAR(1)
    name_prefix: CHAR(4)
    first_name: CHAR(30)
    last_name: CHAR(30)
    street_nbr: SMALLINT
    street_name: CHAR(30)
    postal_code: CHAR(5)
    city_name: CHAR(20)
    state_code: CHAR(2)


In [8]:
SELECT * FROM ${XSPDB}.Accounts SAMPLE 10;

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
1,0000000013631122,1363112,CK,Y,1994-07-08,,2759.11,1085.16
2,0000000013632963,1363296,SV,Y,1991-06-28,,2285.95,2505.34
3,0000000013625723,1362572,SV,Y,1993-11-17,,202.51,432.85
4,4561143213633084,1363308,CC,N,1994-08-10,1995-04-15,2794.47,498.06
5,4561143213626054,1362605,CC,Y,1991-01-11,,705.21,1000.00
6,0000000013634652,1363465,CK,Y,1995-12-29,,2605.30,1632.35
7,0000000013633213,1363321,SV,Y,1994-07-23,,172.70,1009.61
8,0000000013631943,1363194,SV,N,1990-09-25,1995-04-21,210.50,455.67
9,0000000013628483,1362848,SV,Y,1990-08-07,,5265.17,4844.42
10,0000000013625622,1362562,CK,Y,1995-11-17,,7997.57,5521.65


In [9]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2021.04.02_21.36.10.752_UTC
History ID:    937
Rows:          10 of 10
Parts:         2
Column Definitions:
    acct_nbr: CHAR(16)
    cust_id: INTEGER
    acct_type: CHAR(2)
    account_active: CHAR(1)
    acct_start_date: DATE
    acct_end_date: DATE
    starting_balance: DECIMAL(9, 2)
    ending_balance: DECIMAL(9, 2)


In [10]:
SELECT * FROM ${XSPDB}.Transactions SAMPLE 10;

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
1,1,4561143213634554,-157.23,-157.23,.00,-157.23,1995-11-20,54727,E,CG
2,80,0000000013633352,-.15,-.15,.00,696.63,1995-09-13,0,,FK
3,27,4561143213629454,288.66,286.91,1.75,-432.77,1995-08-01,0,M,PM
4,50,0000000013625732,593.22,593.22,.00,6891.60,1995-02-09,103708,B,DP
5,31,4561143213632004,78.07,74.33,3.74,-191.39,1995-09-07,0,M,PM
6,81,0000000013628772,.00,.00,.00,18.34,1995-03-12,140405,A,IQ
7,6,0000000013627362,3.46,.00,3.46,2770.29,1995-01-31,235959,,IN
8,10,0000000013634052,-171.19,-171.19,.00,143.33,1995-03-18,193003,V,TR
9,35,4561143213626634,333.41,287.99,45.42,-4112.01,1995-08-05,0,M,PM
10,14,0000000013631572,-51.01,-51.01,.00,9480.22,1995-12-23,0,P,WD


In [11]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2021.04.02_21.36.10.878_UTC
History ID:    938
Rows:          10 of 10
Parts:         2
Column Definitions:
    tran_id: INTEGER
    acct_nbr: CHAR(16)
    tran_amt: DECIMAL(9, 2)
    principal_amt: DECIMAL(9, 2)
    interest_amt: DECIMAL(9, 2)
    new_balance: DECIMAL(9, 2)
    tran_date: DATE
    tran_time: INTEGER
    channel: CHAR(1)
    tran_code: CHAR(2)


Criamos o seguinte Data Set Analítico (ADS), por meio do Join dessas 3 tabelas:

In [12]:
CREATE TABLE ${XSPDB}.VAL_ADS AS (
    SELECT 
        T1.cust_id  AS cust_id
       ,MIN(T1.income) AS tot_income
       ,MIN(T1.age) AS tot_age
       ,MIN(T1.years_with_bank) AS tot_cust_years
       ,MIN(T1.nbr_children) AS tot_children
       ,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind
       ,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind
       ,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind
       ,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind
       ,MAX(CASE WHEN T1.state_code = 'CA' THEN 1 ELSE 0 END) AS ca_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'NY' THEN 1 ELSE 0 END) AS ny_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'TX' THEN 1 ELSE 0 END) AS tx_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'IL' THEN 1 ELSE 0 END) AS il_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'AZ' THEN 1 ELSE 0 END) AS az_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'OH' THEN 1 ELSE 0 END) AS oh_resident_ind
       ,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt
    FROM ${XSPDB}.Customer AS T1
        LEFT OUTER JOIN ${XSPDB}.Accounts AS T2
            ON T1.cust_id = T2.cust_id
        LEFT OUTER JOIN ${XSPDB}.Transactions AS T3
            ON T2.acct_nbr = T3.acct_nbr
GROUP BY T1.cust_id) WITH DATA UNIQUE PRIMARY INDEX (cust_id);

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3803:Table 'VAL_ADS' already exists.

In [13]:
SELECT * FROM ${XSPDB}.VAL_ADS SAMPLE 10;

,cust_id,tot_income,tot_age,tot_cust_years,tot_children,single_ind,female_ind,married_ind,separated_ind,ca_resident_ind,ny_resident_ind,tx_resident_ind,il_resident_ind,az_resident_ind,oh_resident_ind,ck_acct_ind,sv_acct_ind,cc_acct_ind,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
1,1363035,8262,60,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,363.03830357142857,159.76071428571427,0,0.8545535714285715,0.1994642857142857,0,3,3,10,95
2,1362705,27635,40,0,2,0,1,0,0,0,0,0,0,0,0,1,1,1,1917.1834239130435,284.5704347826087,118.99364130434783,-18.48353260869565,6.489347826086957,-2.245163043478261,50,74,22,38
3,1363174,21384,58,7,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1362668,0,13,5,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1684.58,0,0,31.419,0,6,13,11,10
5,1363133,12823,66,6,0,1,1,0,0,0,0,0,0,0,0,1,1,1,2476.5692063492065,418.69,26.296825396825398,-26.482857142857142,14.422698412698413,26.60142857142857,18,13,13,19
6,1362594,69205,53,3,2,0,0,1,0,0,0,1,0,0,0,1,1,1,9657.341151832461,1711.9935078534031,2701.979057591623,-51.278848167539266,24.26523560209424,54.66,39,75,71,6
7,1363259,7335,28,7,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,714.9,0,0,0.9396363636363636,13,10,15,17
8,1362935,0,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1362962,2858,83,3,0,0,0,0,0,1,0,0,0,0,0,1,0,1,360.65289940828404,0,271.1733727810651,0.019644970414201185,0,-3.338224852071006,116,17,16,20
10,1362541,22362,63,3,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,2093.96,0,0,-42.63875,1,3,22,22


In [14]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2021.04.02_21.36.11.242_UTC
History ID:    940
Rows:          10 of 10
Parts:         2
Column Definitions:
    cust_id: INTEGER
    tot_income: INTEGER
    tot_age: SMALLINT
    tot_cust_years: SMALLINT
    tot_children: SMALLINT
    single_ind: BYTEINT
    female_ind: BYTEINT
    married_ind: BYTEINT
    separated_ind: BYTEINT
    ca_resident_ind: BYTEINT
    ny_resident_ind: BYTEINT
    tx_resident_ind: BYTEINT
    il_resident_ind: BYTEINT
    az_resident_ind: BYTEINT
    oh_resident_ind: BYTEINT
    ck_acct_ind: BYTEINT
    sv_acct_ind: BYTEINT
    cc_acct_ind: BYTEINT
    ck_avg_bal: FLOAT(0, 0)
    sv_avg_bal: FLOAT(0, 0)
    cc_avg_bal: FLOAT(0, 0)
    ck_avg_tran_amt: FLOAT(0, 0)
    sv_avg_tran_amt: FLOAT(0, 0)
    cc_avg_tran_amt: FLOAT(0, 0)
    q1_trans_cnt: INTEGER
    q2_trans_cnt: INTEGER
    q3_trans_cnt: INTEGER
    q4_trans_cnt: INTEGER


---

## Regressão Linear



1.  Utilizando o dataset analítico criado (tabela VAL_ADS) vamos construir um modelo linear para estimar o saldo médio mensal (cc_avg_bal) que um cliente bancário tem em seu cartão de crédito com base em todas as variáveis não relacionadas a cartão de crédito na tabela. Os coeficientes e estatísticas de variáveis de modelo são criadas na tabela LinearRegressionDemo1 conforme especificado pelo argumento outputtablename. Observe que as estatísticas do modelo são criadas na tabela LinearRegressionDemo1_rpt.

In [15]:
call ${XSPDB}.td_analyze('linear',
                         'database=${XSPDB};
                          tablename=VAL_ADS;
                          columns=tot_age,tot_income,tot_cust_years,tot_children,single_ind,female_ind,married_ind,separated_ind,ck_acct_ind,sv_acct_ind,sv_avg_bal,ck_avg_bal,ca_resident_ind,ny_resident_ind,tx_resident_ind,il_resident_ind,az_resident_ind,oh_resident_ind;
                          dependent=cc_avg_bal;
                          neardependencyreport=true;
                          outputdatabase=${QLID};
                          outputtablename=LinearRegressionDemo1');

Success: 0 rows affected

In [16]:
SELECT * FROM ${QLID}.LinearRegressionDemo1 order by 2 DESC;

,Column Name,B Coefficient,Standard Error,T Statistic,P-Value,Lower,Upper,Standard Coefficient,Incremental R-Squared,Squared Multiple Correlation Coefficient (1-Tolerance)
1,separated_ind,328.174825176008,169.342592085858,1.93793434441833,0.0530175785931573,-4.28328146168019,660.632931813697,0.0848428323218558,0.130788263183031,0.389976743529633
2,(Constant),141.038765533841,197.290249312991,0.714879554488726,0.474912649405976,-246.286962369389,528.36449343707,0,0,0
3,female_ind,97.6713334824149,66.9278782111552,1.4593520083554,0.144899589774756,-33.7233463632627,229.066013328092,0.051121536217369,0.124543420104367,0.0471798825676981
4,oh_resident_ind,53.9832449662958,193.46160098053,0.279038551798859,0.780294487429917,-325.825973593872,433.792463526464,0.00983202279826291,0.148803225613419,0.058237800709563
5,single_ind,39.4868288639353,145.456747470657,0.271467838725743,0.786108192474717,-246.077920477169,325.05157820504,0.0200940714044542,0.122127025040395,0.786597512132666
6,tot_cust_years,19.447927470585,12.5205484386463,1.55328079803249,0.120790663374464,-5.13276293166995,44.0286178728399,0.054825224612203,0.122047929599545,0.0614926081292312
7,tot_age,2.20096107482116,2.28116113917766,0.964842437923779,0.334944124937623,-2.27747818800746,6.67940033764978,0.0443265890635109,0.0134217828011889,0.446034450278544
8,tot_income,0.0161884534218535,0.00177005055701053,9.14575764954111,0,0.0127134407296045,0.0196634661141025,0.378773848214807,0.118188037392676,0.318324013506843
9,sv_avg_bal,-0.02764065651019,0.0361464293578714,-0.764685668853508,0.444706537857027,-0.0986043362201245,0.0433230231997444,-0.0294395024604728,0.139474213090453,0.211133415835878
10,ck_avg_bal,-0.0369196825088095,0.0178219709569634,-2.07158246402507,0.0386560704406278,-0.0719082736713912,-0.00193109134622772,-0.0849452445117186,0.144454983582387,0.304615669228816


In [17]:
SELECT * FROM ${QLID}.LinearRegressionDemo1_rpt order by 2 DESC;

,rid,Total Observations,Total Sum of Squares,Multiple Correlation Coefficient (R):,Squared Multiple Correlation Coefficient (1-Tolerance),Adjusted R-Squared,Standard Error of Estimate,Regression Sum of Squares,Regression Degrees of Freedom,Regression Mean-Square,Regression F Ratio,Regression P-Value,Residual Sum of Squares,Residual Degrees of Freedom,Residual Mean-Square,Output Database,Output Tablename,Dependent
1,1,747,6.72013534623019e+08,0.385750211423687,0.148803225613419,0.127757151521443,886.417466249104,9.99977816077802e+07,18,5.55543231154335e+06,7.07035549542953,1.11022302462516e-16,5.72015753015239e+08,728,785735.924471482,NOS_USR,VAL_ADS,cc_avg_bal


In [18]:
SELECT * FROM ${QLID}.LinearRegressionDemo1_txt order by 2 DESC;

,partId,XmlModel
1,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS18cc_avg_balIncludeConstantnone"


In [19]:
SELECT XMLSERIALIZE(Content X.Dot) as XMLText
FROM (SELECT * FROM ${QLID}.LinearRegressionDemo1_txt) AS C,
XMLTable (
'//*'
PASSING CREATEXML(C.XmlModel)
) AS X ("Dot");

,XMLText
1,NOS_USRVAL_ADS18cc_avg_balIncludeConstantnone
2,NOS_USRVAL_ADS18cc_avg_balIncludeConstantnone
3,NOS_USR
4,VAL_ADS
5,18
6,cc_avg_bal
7,IncludeConstant
8,none
9,


2)  A fim de mostrar o recurso "GROUP BY" da Regressão Linear, vamos construir um modelo para cada state_code; para fazer isso, o dataset analítico original foi modificado para incluir state_code em vez das variáveis indicadoras dos estados:

In [20]:
CREATE TABLE ${XSPDB}.VAL_ADS2 AS (
    SELECT 
        T1.cust_id  AS cust_id
       ,MIN(T1.income) AS tot_income
       ,MIN(T1.age) AS tot_age
       ,MIN(T1.years_with_bank) AS tot_cust_years
       ,MIN(T1.nbr_children) AS tot_children
       ,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind
       ,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind
       ,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind
       ,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind
       ,MAX(CASE WHEN T1.state_code = 'CA' THEN 'CA'
                 WHEN T1.state_code = 'NY' THEN 'NY'
                 WHEN T1.state_code = 'TX' THEN 'TX'
                 WHEN T1.state_code = 'IL' THEN 'IL'
                 WHEN T1.state_code = 'AZ' THEN 'AZ'
                 WHEN T1.state_code = 'OH' THEN 'OH' ELSE 'OTHER' END) AS state_code
       ,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt
    FROM ${XSPDB}.Customer AS T1
        LEFT OUTER JOIN ${XSPDB}.Accounts AS T2
            ON T1.cust_id = T2.cust_id
        LEFT OUTER JOIN ${XSPDB}.Transactions AS T3
            ON T2.acct_nbr = T3.acct_nbr
GROUP BY T1.cust_id) WITH DATA UNIQUE PRIMARY INDEX (cust_id);

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3803:Table 'VAL_ADS2' already exists.

In [21]:
SELECT * FROM ${XSPDB}.VAL_ADS2 SAMPLE 10;

,cust_id,tot_income,tot_age,tot_cust_years,tot_children,single_ind,female_ind,married_ind,separated_ind,state_code,ck_acct_ind,sv_acct_ind,cc_acct_ind,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
1,1362837,22322,50,0,0,0,0,1,0,AZ,1,1,1,2354.5735714285715,127.79642857142858,125.71428571428571,-16.199,2.6847142857142856,-7.706857142857143,9,12,27,92
2,1362811,8011,82,2,0,0,1,1,0,OTHER,1,0,1,1696.1057142857144,0,455.57660714285714,-13.816428571428572,0,-5.185297619047619,107,25,9,27
3,1362969,75121,58,8,0,0,1,0,0,IL,0,1,1,0,3392.7736082474225,3212.0207216494846,0,48.75515463917526,-9.521237113402062,26,21,25,25
4,1362745,11410,82,2,0,0,0,1,0,OTHER,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1362643,6356,18,3,0,1,1,0,0,NY,1,1,1,3558.552105263158,112.84778947368422,507.4066315789474,-25.539052631578947,3.779578947368421,10.468,34,25,23,13
6,1362980,8201,18,3,0,1,0,0,0,IL,1,1,1,1486.1233678756478,112.3979274611399,173.62259067357513,-13.785751295336787,0.4058031088082902,-2.0901554404145077,114,18,14,47
7,1362939,47267,54,4,1,0,0,0,0,NY,1,1,1,1137.6432692307692,540.5923076923077,2075.6327884615384,4.885576923076923,-2.2274038461538463,22.963701923076922,24,36,114,34
8,1363270,0,16,1,0,1,0,0,0,TX,1,1,0,156.19623287671232,299.10027397260274,0,-1.3632191780821918,-1.1576712328767123,0,102,3,3,38
9,1362650,17804,21,0,1,0,0,1,0,NY,0,1,1,0,199.73586206896553,344.82758620689657,0,-0.11137931034482759,-17.24137931034483,3,9,8,38
10,1363126,58749,50,0,2,0,1,1,0,OTHER,1,1,1,6122.7068023255815,761.0288372093023,691.2790697674419,-32.2878488372093,12.716802325581396,-15.800988372093023,11,24,113,24


In [22]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2021.04.02_21.36.19.051_UTC
History ID:    947
Rows:          10 of 10
Parts:         2
Column Definitions:
    cust_id: INTEGER
    tot_income: INTEGER
    tot_age: SMALLINT
    tot_cust_years: SMALLINT
    tot_children: SMALLINT
    single_ind: BYTEINT
    female_ind: BYTEINT
    married_ind: BYTEINT
    separated_ind: BYTEINT
    state_code: VARCHAR(5)
    ck_acct_ind: BYTEINT
    sv_acct_ind: BYTEINT
    cc_acct_ind: BYTEINT
    ck_avg_bal: FLOAT(0, 0)
    sv_avg_bal: FLOAT(0, 0)
    cc_avg_bal: FLOAT(0, 0)
    ck_avg_tran_amt: FLOAT(0, 0)
    sv_avg_tran_amt: FLOAT(0, 0)
    cc_avg_tran_amt: FLOAT(0, 0)
    q1_trans_cnt: INTEGER
    q2_trans_cnt: INTEGER
    q3_trans_cnt: INTEGER
    q4_trans_cnt: INTEGER


Agora, vamos construir um modelo de regressão linear para cada state_code. Os coeficientes e estatísticas de variáveis são criadas na tabela LinearRegressionDemo2 conforme especificado pelo argumento outputtablename. Observe que as estatísticas do modelo são criadas com a tabela LinearRegressionDemo2_rpt, uma para cada state_code.

In [23]:
call ${XSPDB}.td_analyze('linear',
                         'database=${XSPDB};
                          tablename=VAL_ADS2;
                          columns=tot_age,tot_income,tot_cust_years,tot_children,single_ind,married_ind,separated_ind,female_ind,ck_acct_ind,sv_acct_ind,sv_avg_bal,ck_avg_bal;
                          dependent=cc_avg_bal;
                          outputdatabase=${QLID};
                          outputtablename=LinearRegressionDemo2;
                          groupby=state_code');

Success: 0 rows affected

In [24]:
SELECT * FROM ${QLID}.LinearRegressionDemo2 ORDER BY 1, 2;

,state_code,Column Name,B Coefficient,Standard Error,T Statistic,P-Value,Lower,Upper,Standard Coefficient,Incremental R-Squared,Squared Multiple Correlation Coefficient (1-Tolerance)
1,AZ,(Constant),-399.09910392595,807.553174355432,-0.49420783249908,0.630880192930709,-2176.51165666717,1378.31344881527,0,0,0
2,AZ,ck_acct_ind,19.6571842074301,299.01377199779,0.065740062994742,0.948764355650142,-638.467690622737,677.782059037597,0.0176777249233066,0.332723620873046,0.321677067357512
3,AZ,ck_avg_bal,0.0637643947858046,0.100388326138203,0.635177388036346,0.538300966701435,-0.157188821290822,0.284717610862431,0.221185389395711,0.460462423721281,0.595511482141048
4,AZ,female_ind,241.818516844219,303.329331702009,0.797214418689258,0.442194988663199,-425.804840852426,909.441874540863,0.25242815881451,0.329796252720349,0.510780564596873
5,AZ,married_ind,231.337515030165,443.991169653335,0.521040801804214,0.612666222003621,-745.880460588566,1208.5554906489,0.256135971018127,0.304057724133513,0.797030243473302
6,AZ,separated_ind,49.2832789069808,560.837723396557,0.0878744008311538,0.931555855235292,-1185.11222750843,1283.6787853224,0.0360921813497783,0.305047965393857,0.709244791430716
7,AZ,single_ind,-39.2395923720176,514.017307085815,-0.0763390489602066,0.940520125595204,-1170.58405729818,1092.10487255414,-0.0394949191859254,0.267252493672782,0.816751662688792
8,AZ,sv_acct_ind,-353.534424863496,353.51387831731,-1.00005812090401,0.338773801194839,-1131.61322492634,424.544375199353,-0.355834826392935,0.336411310753914,0.612579505869209
9,AZ,sv_avg_bal,0.21018575008192,0.13223896484196,1.58943886420401,0.140268639412896,-0.0808702491211182,0.501241749284957,0.481941609849692,0.440673641474775,0.466508112904138
10,AZ,tot_age,10.9680673369554,11.57184181197,0.947823822272612,0.363576958059713,-14.5013847661187,36.4375194400295,0.356009646172654,0.0914661466440459,0.652335201208369


In [25]:
SELECT * FROM ${QLID}.LinearRegressionDemo2_rpt ORDER BY 1, 2;

,rid,state_code,Total Observations,Total Sum of Squares,Multiple Correlation Coefficient (R):,Squared Multiple Correlation Coefficient (1-Tolerance),Adjusted R-Squared,Standard Error of Estimate,Regression Sum of Squares,Regression Degrees of Freedom,Regression Mean-Square,Regression F Ratio,Regression P-Value,Residual Sum of Squares,Residual Degrees of Freedom,Residual Mean-Square,Output Database,Output Tablename,Dependent
1,1,AZ,24,4.89443127840392e+06,0.67857381597088,0.460462423721282,-0.128124023128229,489.965637868691,2.25370168919112e+06,12,187808.47409926,0.782319107390205,0.661219184374013,2.6407295892128e+06,11,240066.326292073,NOS_USR,VAL_ADS2,cc_avg_bal
2,2,CA,177,2.0861452150148e+08,0.517933772968521,0.268255393181407,0.214713104877608,964.784036913244,5.59619704887305e+07,12,4.66349754072755e+06,5.01015929052795,4.59834429422656e-07,1.52652551012749e+08,164,930808.237882616,NOS_USR,VAL_ADS2,cc_avg_bal
3,3,IL,56,3.37652557123531e+07,0.658352657409312,0.433428221517903,0.275315167057782,667.003730331097,1.46348147325024e+07,12,1.2195678943752e+06,2.74125512910905,0.00755479438379436,1.91304409798507e+07,43,444893.976275598,NOS_USR,VAL_ADS2,cc_avg_bal
4,4,NY,107,6.40704308897399e+07,0.462089029255775,0.213526270958545,0.113125369378784,732.161698097581,1.36807201865933e+07,12,1.14006001554944e+06,2.1267365890028,0.022053217092853,5.03897107031466e+07,94,536060.752161134,NOS_USR,VAL_ADS2,cc_avg_bal
5,5,OH,23,9.29443841076735e+06,,,,,,,,,,,,,NOS_USR,VAL_ADS2,cc_avg_bal
6,6,OTHER,277,3.10330884847684e+08,0.434794289217757,0.189046073936375,0.152184531842574,976.35696062879,5.86668354016559e+07,12,4.88890295013799e+06,5.12854490610597,1.03767098025287e-07,2.51664049446028e+08,264,953272.914568288,NOS_USR,VAL_ADS2,cc_avg_bal
7,7,TX,83,3.91038748185278e+07,0.485821176169109,0.236022215214336,0.105054594965365,653.282827182342,9.22938315813302e+06,12,769115.263177751,1.80214174133771,0.0644816315921763,2.98744916603947e+07,70,426778.452291353,NOS_USR,VAL_ADS2,cc_avg_bal


In [26]:
SELECT * FROM ${QLID}.LinearRegressionDemo2_txt ORDER BY 1, 2;

,state_code,partId,XmlModel
1,AZ,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"
2,CA,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"
3,IL,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"
4,NY,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"
5,OH,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnoneConstant columns detected...run terminated."
6,OTHER,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"
7,TX,1,"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?>NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone"


In [27]:
SELECT XMLSERIALIZE(Content X.Dot) as XMLText
FROM (SELECT * FROM ${QLID}.LinearRegressionDemo2_txt) AS C,
XMLTable (
'//*'
PASSING CREATEXML(C.XmlModel)
) AS X ("Dot");

,XMLText
1,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
2,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
3,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnoneConstant columns detected...run terminated.
4,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
5,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
6,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
7,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
8,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnoneConstant columns detected...run terminated.
9,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone
10,NOS_USRVAL_ADS212cc_avg_balIncludeConstantnone


---

## Scoring da Regressão Linear

### Objetivo

O Scoring da Regressão Linear é a aplicação de um modelo de regressão linear a uma tabela de entrada que contém as mesmas colunas de variáveis independentes contidas no modelo. O resultado é uma tabela de Scoring de saída que contém, no mínimo, uma ou mais colunas-chave e uma estimativa da variável dependente no modelo. O usuário também pode optar por realizar a avaliação do modelo, separadamente ou em combinação com o Scoring. Quando solicitado, um relatório é produzido como um conjunto de dados de resultados contendo o erro padrão da estimativa, bem como o erro absoluto mínimo, máximo e médio. Quando a avaliação do modelo é solicitada, a tabela de entrada deve
contém uma coluna que representa a variável dependente no modelo. Quando tanto o Scoring quanto a avaliação são solicitadas, a tabela de saída inclui automaticamente o valor residual, calculado como a diferença entre o valor original e o valor previsto da variável dependente. O valor residual também pode ser solicitado quando apenas o Scoring é realizado.



---

1) Primeiramente, vamos executar o Scoring do modelo de regressão linear criado acima - para fins de demonstração, usaremos a mesma tabela VAL_ADS para o Scoring. O erro absoluto mínimo, máximo e médio e o erro padrão da estimativa são retornados como um conjunto de resultados. O conjunto de dados pós-Scoring inclui o real junto com a previsão e o residual que estamos avaliando também.

In [28]:
call ${XSPDB}.td_analyze('linearscore',
                         'database=${XSPDB};
                          tablename=VAL_ADS;
                          modeldatabase=${QLID};
                          modeltablename=LinearRegressionDemo1;
                          outputdatabase=${QLID};
                          outputtablename=LinearRegressionScore1;
                          predicted=estimate;
                          retain=cc_avg_bal;
                          scoringmethod=scoreandevaluate;');

Success: 0 rows affected

In [29]:
SELECT * FROM ${QLID}.LinearRegressionScore1;

,cust_id,cc_avg_bal,estimate,Residual
1,1363160,553.4672413793104,757.4626521344095,-203.99541075509907
2,1362813,0,567.6544791988415,-567.6544791988415
3,1362691,0,709.6684461677008,-709.6684461677008
4,1362588,421.9693548387097,305.28982833796016,116.67952650074953
5,1362752,287.7675,1024.122214403084,-736.3547144030839
6,1363261,317.9313368983957,535.3765804780354,-217.4452435796397
7,1362487,481.00349514563106,103.68281838174167,377.3206767638894
8,1362548,1163.5072180451127,924.8430201784508,238.66419786666194
9,1362996,0,695.1770295288964,-695.1770295288964
10,1363078,1009.2228571428572,307.26944186890796,701.9534152739492


2) Em seguida, vamos executar o Scoring dos modelos de regressão linear dos diversos state_code novamente usando a tabela VAL_ADS2. Para cada modelo, o erro absoluto mínimo, máximo e médio e o erro padrão da estimativa são retornados como um conjunto de resultados. O conjunto de dados pós-Scoring inclui o real junto com o state_code, a predição e o residual que estamos avaliando também.

In [30]:
call ${XSPDB}.td_analyze('linearscore',
                         'database=${XSPDB};
                          tablename=VAL_ADS2;
                          modeldatabase=${QLID};
                          modeltablename=LinearRegressionDemo2;
                          outputdatabase=${QLID};
                          outputtablename=LinearRegressionScore2;
                          predicted=estimate;
                          retain=cc_avg_bal;
                          scoringmethod=scoreandevaluate;');

Success: 0 rows affected

In [31]:
SELECT * FROM ${QLID}.LinearRegressionScore2;

,cust_id,state_code,cc_avg_bal,estimate,Residual
1,1363160,CA,553.4672413793104,952.9294944568617,-399.4622530775513
2,1362813,IL,0,760.5849494006503,-760.5849494006503
3,1362691,OTHER,0,756.4062612226761,-756.4062612226761
4,1362588,OTHER,421.9693548387097,269.4058033604687,152.56355147824098
5,1362752,CA,287.7675,1322.2094156863052,-1034.441915686305
6,1363261,NY,317.9313368983957,521.9640115676227,-204.032674669227
7,1362487,CA,481.00349514563106,-38.336865708376294,519.3403608540074
8,1362548,NY,1163.5072180451127,1385.4601749530248,-221.9529569079121
9,1362996,OTHER,0,600.9025290538956,-600.9025290538956
10,1363078,OTHER,1009.2228571428572,431.9215287585205,577.3013283843367


---

---

## Regressão Logística


---

1) Mais uma vez usando a tabela VAL_ADS, vamos construir um modelo de regressão logística para prever a propensão da base de clientes para abrir uma conta de cartão de crédito (cc_acct_ind) com base em todas as variáveis que não são de cartão de crédito no conjunto de dados analíticos. Os coeficientes do modelo e as estatísticas de variáveis são criados no outputtablename especificado. Além disso, as estatísticas do modelo são criadas dentro de uma tabela com uma extensão "_rpt" no outputtablename. Os relatórios para successtabpe, thresholdtable e lifttable são retornados em uma string XML dentro de uma tabela com uma extensão "_txt" no outputtablename.

In [32]:
call ${XSPDB}.td_analyze('logistic',
                         'database=${XSPDB};
                          tablename=VAL_ADS;
                          columns=tot_age,tot_income,tot_cust_years,tot_children,single_ind,married_ind,separated_ind,female_ind,ck_acct_ind,sv_acct_ind,sv_avg_bal,ck_avg_bal,ca_resident_ind,ny_resident_ind,tx_resident_ind,il_resident_ind,az_resident_ind,oh_resident_ind;
                          dependent=cc_acct_ind;
                          outputdatabase=${QLID};
                          outputtablename=LogisticOut1;
                          statstable=true;
                          successtable=true;
                          thresholdtable=true;
                          lifttable=true;');

Success: 0 rows affected

In [33]:
SELECT * FROM ${QLID}.LogisticOut1 ORDER BY 2 DESC;

,Column Name,B Coefficient,Standard Error,Wald Statistic,T Statistic,P-Value,Odds Ratio,Lower,Upper,Partial R,Standardized Coefficient
1,ck_acct_ind,1.52093642910134,0.21122470736138,51.8480813763378,7.2005611848201,1.49813494942919e-12,4.57650876518957,3.0250983701987,6.92355418395246,0.224707462646606,0.385928447963603
2,sv_acct_ind,1.00749545826154,0.194358930436256,26.8705894294096,5.18368492767544,2.82103366222586e-07,2.73873314669911,1.87116073935178,4.00855954867182,0.15872170177561,0.275660205366892
3,separated_ind,0.875270852091336,0.5348948201149,2.67761509418834,1.63634198570725,0.102200240052689,2.39952512235704,0.841050367449314,6.84586920790991,0.0261990056146459,0.118408479127344
4,oh_resident_ind,0.474759765128598,0.531936018276908,0.796579299625616,0.892512912862115,0.37241298999707,1.60762794279962,0.56676208143691,4.56005736290215,0,0.0452468383275301
5,ny_resident_ind,0.368567190559411,0.276918289745215,1.77145474884433,1.33096008536858,0.183618823394613,1.44566177218872,0.840143402879551,2.48759670361592,0,0.0712327446580372
6,female_ind,0.344845850408636,0.175064391622608,3.88020232073237,1.96982291608468,0.0492373549559142,1.41177227877359,1.00172743815818,1.98966394569184,0.0436410875067018,0.0944478822970212
7,single_ind,0.287476775640691,0.39671872583054,0.525097716185238,0.72463626474614,0.468908028454297,1.33305963234781,0.612578832342531,2.90092946339619,0,0.0765506513068479
8,tot_children,0.116107049904922,0.105336704651888,1.21494809767887,1.10224684063002,0.270718632837568,1.12311609516941,0.913612189896651,1.3806621421845,0,0.0706327842079023
9,il_resident_ind,0.0615990756599514,0.347565379755017,0.031410537563781,0.1772301824289,0.859376885808156,1.06353586182654,0.538150751440337,2.10184325927959,0,0.00894928549392125
10,tot_cust_years,0.0321964569098754,0.0336141845271347,0.917425916218723,0.957823530833693,0.338469725453793,1.03272037043008,0.966874926055157,1.10304997550469,0,0.0474947511684139


In [34]:
SELECT * FROM ${QLID}.LogisticOut1_rpt ORDER BY 1;

,rid,Total Observations,Total Iterations,Initial Log Likelihood,Final Log Likelihood,Likelihood Ratio Test G Statistic,Chi-Square Degrees of Freedom,Chi-Square Value,Chi-Square Probability,McFaddens Pseudo R-Squared,Dependent Variable,Dependent Response Value,Total Distinct Values
1,1,747,5,-493.609443676595,-407.646702345764,171.925482661661,18,28.8692994303784,0,0.174151330433524,cc_acct_ind,1,2


In [35]:
SELECT * FROM ${QLID}.LogisticOut1_txt ORDER BY 1;

partId XmlModel 1 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS 18 cc_acct_ind 1 none 0 false IncludeConstant tot_age 42.479250334672024 19.11487876817342 tot_income 22728.281124497993 22207.221405394437 tot_cust_years 3.9076305220883536 2.675633899491365 tot_children 0.714859437751004 1.1034099354173406 single_ind 0.36947791164658633 0.4829868203308733 married_ind 0.4725568942436412 0.49958081132105875 separated_ind 0.0642570281124498 0.24537453631237413 female_ind 0.5595716198125836 0.49677116027545415 ck_acct_ind 0.6961178045515395 0.4602405561024589 sv_acct_ind 0.5635876840696118 0.4962724120676887 sv_avg_bal 385.83482311918493 1010.8848973552347 ck_avg_bal 1321.9485572643352 2183.7388946453607 ca_resident_ind 0.23694779116465864 0.42549488764890675 ny_resident_ind 0.14323962516733602 0.3505517675016328 tx_resident_ind 0.1111111111111111 0.31448024650108053 il_resident_ind 0.0749665327978581 0.2635137749763863 az_resident_ind 0.0321285140562249 0.17645950428454302 oh_resident_ind 0.030789825970548863 0.17286357342422898 cc_acct_ind 0.6265060240963856 0.48405567213949435

In [36]:
SELECT XMLSERIALIZE(Content X.Dot) as XMLText
FROM (SELECT * FROM ${QLID}.LogisticOut1_txt) AS C,
XMLTable (
'//*'
PASSING CREATEXML(C.XmlModel)
) AS X ("Dot");

XMLText 1 NOS_USR VAL_ADS 18 cc_acct_ind 1 none 0 false IncludeConstant tot_age 42.479250334672024 19.11487876817342 tot_income 22728.281124497993 22207.221405394437 tot_cust_years 3.9076305220883536 2.675633899491365 tot_children 0.714859437751004 1.1034099354173406 single_ind 0.36947791164658633 0.4829868203308733 married_ind 0.4725568942436412 0.49958081132105875 separated_ind 0.0642570281124498 0.24537453631237413 female_ind 0.5595716198125836 0.49677116027545415 ck_acct_ind 0.6961178045515395 0.4602405561024589 sv_acct_ind 0.5635876840696118 0.4962724120676887 sv_avg_bal 385.83482311918493 1010.8848973552347 ck_avg_bal 1321.9485572643352 2183.7388946453607 ca_resident_ind 0.23694779116465864 0.42549488764890675 ny_resident_ind 0.14323962516733602 0.3505517675016328 tx_resident_ind 0.1111111111111111 0.31448024650108053 il_resident_ind 0.0749665327978581 0.2635137749763863 az_resident_ind 0.0321285140562249 0.17645950428454302 oh_resident_ind 0.030789825970548863 0.17286357342422898 cc_acct_ind 0.6265060240963856 0.48405567213949435 2 NOS_USR VAL_ADS 18 cc_acct_ind 1 none 0 false IncludeConstant tot_age 42.479250334672024 19.11487876817342 tot_income 22728.281124497993 22207.221405394437 tot_cust_years 3.9076305220883536 2.675633899491365 tot_children 0.714859437751004 1.1034099354173406 single_ind 0.36947791164658633 0.4829868203308733 married_ind 0.4725568942436412 0.49958081132105875 separated_ind 0.0642570281124498 0.24537453631237413 female_ind 0.5595716198125836 0.49677116027545415 ck_acct_ind 0.6961178045515395 0.4602405561024589 sv_acct_ind 0.5635876840696118 0.4962724120676887 sv_avg_bal 385.83482311918493 1010.8848973552347 ck_avg_bal 1321.9485572643352 2183.7388946453607 ca_resident_ind 0.23694779116465864 0.42549488764890675 ny_resident_ind 0.14323962516733602 0.3505517675016328 tx_resident_ind 0.1111111111111111 0.31448024650108053 il_resident_ind 0.0749665327978581 0.2635137749763863 az_resident_ind 0.0321285140562249 0.17645950428454302 oh_resident_ind 0.030789825970548863 0.17286357342422898 cc_acct_ind 0.6265060240963856 0.48405567213949435 3 NOS_USR 4 VAL_ADS 5 18 6 cc_acct_ind 7 1 8 none 9 0 10 false 11 IncludeConstant 12 tot_age 42.479250334672024 19.11487876817342 tot_income 22728.281124497993 22207.221405394437 tot_cust_years 3.9076305220883536 2.675633899491365 tot_children 0.714859437751004 1.1034099354173406 single_ind 0.36947791164658633 0.4829868203308733 married_ind 0.4725568942436412 0.49958081132105875 separated_ind 0.0642570281124498 0.24537453631237413 female_ind 0.5595716198125836 0.49677116027545415 ck_acct_ind 0.6961178045515395 0.4602405561024589 sv_acct_ind 0.5635876840696118 0.4962724120676887 sv_avg_bal 385.83482311918493 1010.8848973552347 ck_avg_bal 1321.9485572643352 2183.7388946453607 ca_resident_ind 0.23694779116465864 0.42549488764890675 ny_resident_ind 0.14323962516733602 0.3505517675016328 tx_resident_ind 0.1111111111111111 0.31448024650108053 il_resident_ind 0.0749665327978581 0.2635137749763863 az_resident_ind 0.0321285140562249 0.17645950428454302 oh_resident_ind 0.030789825970548863 0.17286357342422898 cc_acct_ind 0.6265060240963856 0.48405567213949435 13 tot_age 42.479250334672024 19.11487876817342 14 tot_age 15 42.479250334672024 16 19.11487876817342 17 tot_income 22728.281124497993 22207.221405394437 18 tot_income 19 22728.281124497993 20 22207.221405394437 21 tot_cust_years 3.9076305220883536 2.675633899491365 22 tot_cust_years 23 3.9076305220883536 24 2.675633899491365 25 tot_children 0.714859437751004 1.1034099354173406 26 tot_children 27 0.714859437751004 28 1.1034099354173406 29 single_ind 0.36947791164658633 0.4829868203308733 30 single_ind 31 0.36947791164658633 32 0.4829868203308733 33 married_ind 0.4725568942436412 0.49958081132105875 34 married_ind 35 0.4725568942436412 36 0.49958081132105875 37 separated_ind 0.0642570281124498 0.24537453631237413 38 separated_ind 39 0.0642570281124498 40 0.24537453631237413 41 female_ind 0.5595716198125836 0.49677116027545415

2) Agora, usando a tabela VAL_ADS2, vamos construir um modelo de regressão logística para prever a propensão da base de clientes para abrir uma conta de cartão de crédito (cc_acct_ind) com base em todas as variáveis que não são de cartão de crédito no conjunto de dados analíticos, criando um modelo para cada state_code usando a opção GROUPBY = state_code. Mais uma vez, os coeficientes do modelo e as estatísticas da variável são criados no outputtablename especificado para cada modelo state_code. Além disso, as estatísticas do modelo são criadas dentro de uma tabela com uma extensão "_rpt" no outputtablename. Os relatórios para successtabpe, thresholdtable e lifttable são retornados em uma string XML dentro de uma tabela com uma extensão "_txt" no outputtablename.

In [37]:
call ${XSPDB}.td_analyze('logistic',
                         'database=${XSPDB};
                          tablename=VAL_ADS2;
                          columns=tot_age,tot_income,tot_cust_years,tot_children,single_ind,married_ind,separated_ind,female_ind,ck_acct_ind,sv_acct_ind,sv_avg_bal,ck_avg_bal;
                          dependent=cc_acct_ind;
                          outputdatabase=${QLID};
                          outputtablename=LogisticOut2;
                          statstable=true;
                          successtable=true;
                          thresholdtable=true;
                          lifttable=true;
                          groupby=state_code');

Success: 0 rows affected

In [38]:
SELECT * FROM ${QLID}.LogisticOut2 ORDER BY 1,3;

,state_code,Column Name,B Coefficient,Standard Error,Wald Statistic,T Statistic,P-Value,Odds Ratio,Lower,Upper,Partial R,Standardized Coefficient
1,AZ,(Constant),-222.177333380171,2741.24527266601,0.00656906725816568,-0.0810497825917237,0.936858240434374,,,,,
2,AZ,tot_children,-73.2981397159156,874.30168286143,0.00702851015092634,-0.0838362102609984,0.934692924735113,1.46900200181085e-32,0,,0,-31.3403109082875
3,AZ,sv_acct_ind,-40.9050539756336,723.008774047522,0.00320086092470019,-0.0565761515543448,0.955897426635008,1.71854447699063e-18,0,,0,-10.4710867331368
4,AZ,tot_income,-0.000571943999680645,0.0183051057261416,0.000976253340716887,-0.0312450530599147,0.975633823668582,0.999428219529111,0.964206985706854,1.03593603945824,0,-6.16388849426798
5,AZ,ck_avg_bal,0.00428472111672978,0.18592340693275,0.000531100832095584,0.0230456250098708,0.982026620171628,1.00429391365872,0.697593683981159,1.44583629148683,0,3.7800609135879
6,AZ,sv_avg_bal,0.0338129422350836,0.412603355228023,0.00671584114063496,0.0819502357570432,0.936158444660651,1.03439109774321,0.460761476743513,2.32216666777898,0,19.7184222430546
7,AZ,tot_age,2.06221659283055,26.6544726951158,0.00598588492065426,0.0773685008298226,0.939719756301197,7.86338041973609,1.61168808445564e-22,3.83652098826387e+23,0,17.0240928322785
8,AZ,tot_cust_years,7.53801718492202,87.3989838068668,0.00743877598280863,0.0862483390147812,0.932818919010359,1878.10240103055,7.57825781581649e-72,4.65445847117386e+77,0,12.7051908540994
9,AZ,female_ind,10.8079622892211,760.468596840199,0.00020198776875043,0.0142122401031797,0.988915150960713,49412.6769769253,0,,0,2.86939350989931
10,AZ,single_ind,30.9771356695184,1383.55479503074,0.000501290862003708,0.02238952572083,0.982538220132627,2.83922026697932e+13,0,,0,7.92968699009396


In [39]:
SELECT * FROM ${QLID}.LogisticOut2_rpt ORDER BY 1;

,rid,state_code,Total Observations,Total Iterations,Initial Log Likelihood,Final Log Likelihood,Likelihood Ratio Test G Statistic,Chi-Square Degrees of Freedom,Chi-Square Value,Chi-Square Probability,McFaddens Pseudo R-Squared,Dependent Variable,Dependent Response Value,Total Distinct Values
1,1,AZ,24,14,-15.8775177157916,-0.000216312526332766,31.7546028065305,12,21.0260698174829,0.00150966916272521,0.999986376174777,cc_acct_ind,1,2
2,2,CA,177,6,-120.619667195349,-92.173936126782,56.8914621371336,12,21.0260698174829,8.26893086225411e-08,0.235829958165095,cc_acct_ind,1,2
3,3,IL,56,10,-37.047541336847,-25.6980241897872,22.6990342941195,12,21.0260698174829,0.030392464366384202,0.306350077158067,cc_acct_ind,1,2
4,4,NY,107,6,-66.1071188619215,-53.5688071505545,25.0766234227342,12,21.0260698174829,0.0144639245917146,0.189666588519097,cc_acct_ind,1,2
5,5,OH,23,,,,,,,,,,,
6,6,OTHER,277,5,-180.578512098577,-143.543527623196,74.0699689507629,12,21.0260698174829,5.50558487688591e-11,0.205090761048935,cc_acct_ind,1,2
7,7,TX,83,5,-56.5089397714094,-45.8826898970036,21.2524997488116,12,21.0260698174829,0.0468030141381988,0.188045465326216,cc_acct_ind,1,2


In [40]:
SELECT * FROM ${QLID}.LogisticOut2_txt ORDER BY 1;

state_code partId XmlModel 1 AZ 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 47.875 14.973345883820414 tot_income 23217.0 19547.463934251664 tot_cust_years 3.2916666666666665 3.0571252006997724 tot_children 0.4166666666666667 0.7755316082290384 single_ind 0.2916666666666667 0.4643056214875365 married_ind 0.5 0.5107539184552492 separated_ind 0.125 0.3378319623460881 female_ind 0.6666666666666666 0.48154341234307685 ck_acct_ind 0.7916666666666666 0.41485111699905347 sv_acct_ind 0.7083333333333334 0.4643056214875365 sv_avg_bal 419.7639597200062 1057.738603128528 ck_avg_bal 1207.6404823688806 1600.167308125376 cc_acct_ind 0.625 0.49453535504684026 2 CA 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 38.898305084745765 17.830669679006707 tot_income 22741.237288135595 21904.39975107143 tot_cust_years 4.0790960451977405 2.6935240681114463 tot_children 0.8870056497175142 1.2830629922234347 single_ind 0.4124293785310734 0.49366817715141204 married_ind 0.4067796610169492 0.4926266819466781 separated_ind 0.05084745762711865 0.22030934967258814 female_ind 0.6045197740112994 0.49034069893006166 ck_acct_ind 0.6666666666666666 0.47274184126735436 sv_acct_ind 0.519774011299435 0.5010261683447327 sv_avg_bal 488.25422400742315 1330.3328843582656 ck_avg_bal 1306.9583035288686 1941.520338111027 cc_acct_ind 0.576271186440678 0.49555030810024064 3 IL 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 39.839285714285715 19.676267925613317 tot_income 27899.803571428572 29764.716274273254 tot_cust_years 3.5535714285714284 2.675951099275599 tot_children 0.8928571428571429 1.038855514121564 single_ind 0.35714285714285715 0.4834937784152282 married_ind 0.44642857142857145 0.5016207497934835 separated_ind 0.08928571428571429 0.28773635118317564 female_ind 0.5178571428571429 0.5042031130613726 ck_acct_ind 0.7321428571428571 0.4468504500113261 sv_acct_ind 0.5178571428571429 0.5042031130613726 sv_avg_bal 588.5857562765214 1538.3772829943812 ck_avg_bal 2133.470746955862 4196.524665266171 cc_acct_ind 0.625 0.48850421045919723 4 NY 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 44.467289719626166 19.603772142312902 tot_income 23650.906542056076 20500.314193879738 tot_cust_years 3.878504672897196 2.539165961617778 tot_children 0.6448598130841121 1.0302256251194661 single_ind 0.34579439252336447 0.47786482138302394 married_ind 0.5233644859813084 0.5018041862802316 separated_ind 0.037383177570093455 0.19059161343186776 female_ind 0.5514018691588785 0.49969131716421034 ck_acct_ind 0.6542056074766355 0.47786482138302394 sv_acct_ind 0.5981308411214953 0.49258297039087456 sv_avg_bal 390.5539574324835 1024.2662121157823 ck_avg_bal 1165.5282995516789 1643.254354994322 cc_acct_ind 0.6915887850467289 0.46401071379284736 5 OH 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant Constant columns detected...run terminated. 6 OTHER 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 43.75812274368231 19.540258908271166 tot_income 20755.7797833935 19896.11042131052 tot_cust_years 4.03971119133574 2.7482294543357635 tot_children 0.6931407942238267 1.1048591088437563 single_ind 0.36101083032490977 0.4811629818467651 married_ind 0.48736462093862815 0.5007450108041379 separated_ind 0.07942238267148015 0.27088627741969934 female_ind 0.555956678700361 0.4977582771429234 ck_acct_ind 0.7003610108303249 0.458928982771801 sv_acct_ind 0.5631768953068592 0.49689034574654706 sv_avg_bal 303.966184415435 679.0307677517072 ck_avg_bal 1204.600106206975 1983.0828939335584 cc_acct_ind 0.6425992779783394 0.48010162163955045 7 TX 1 <?xml vers

In [41]:
SELECT XMLSERIALIZE(Content X.Dot) as XMLText
FROM (SELECT * FROM ${QLID}.LogisticOut2_txt) AS C,
XMLTable (
'//*'
PASSING CREATEXML(C.XmlModel)
) AS X ("Dot")
;

XMLText 1 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 39.839285714285715 19.676267925613317 tot_income 27899.803571428572 29764.716274273254 tot_cust_years 3.5535714285714284 2.675951099275599 tot_children 0.8928571428571429 1.038855514121564 single_ind 0.35714285714285715 0.4834937784152282 married_ind 0.44642857142857145 0.5016207497934835 separated_ind 0.08928571428571429 0.28773635118317564 female_ind 0.5178571428571429 0.5042031130613726 ck_acct_ind 0.7321428571428571 0.4468504500113261 sv_acct_ind 0.5178571428571429 0.5042031130613726 sv_avg_bal 588.5857562765214 1538.3772829943812 ck_avg_bal 2133.470746955862 4196.524665266171 cc_acct_ind 0.625 0.48850421045919723 2 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 44.467289719626166 19.603772142312902 tot_income 23650.906542056076 20500.314193879738 tot_cust_years 3.878504672897196 2.539165961617778 tot_children 0.6448598130841121 1.0302256251194661 single_ind 0.34579439252336447 0.47786482138302394 married_ind 0.5233644859813084 0.5018041862802316 separated_ind 0.037383177570093455 0.19059161343186776 female_ind 0.5514018691588785 0.49969131716421034 ck_acct_ind 0.6542056074766355 0.47786482138302394 sv_acct_ind 0.5981308411214953 0.49258297039087456 sv_avg_bal 390.5539574324835 1024.2662121157823 ck_avg_bal 1165.5282995516789 1643.254354994322 cc_acct_ind 0.6915887850467289 0.46401071379284736 3 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant Constant columns detected...run terminated. 4 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 43.24096385542169 19.96557425062485 tot_income 26229.13253012048 27589.10398114121 tot_cust_years 3.63855421686747 2.4672405502910117 tot_children 0.5783132530120482 0.9255253212030826 single_ind 0.3614457831325301 0.4833398353280309 married_ind 0.5060240963855421 0.5030030300035687 separated_ind 0.060240963855421686 0.23937915584427602 female_ind 0.4819277108433735 0.5027108411726374 ck_acct_ind 0.7349397590361446 0.44404837545182463 sv_acct_ind 0.6385542168674698 0.4833398353280309 sv_avg_bal 363.49111871596347 764.5008656211836 ck_avg_bal 1466.8654516250758 2239.318604633898 cc_acct_ind 0.5783132530120482 0.49683097993894637 5 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 43.75812274368231 19.540258908271166 tot_income 20755.7797833935 19896.11042131052 tot_cust_years 4.03971119133574 2.7482294543357635 tot_children 0.6931407942238267 1.1048591088437563 single_ind 0.36101083032490977 0.4811629818467651 married_ind 0.48736462093862815 0.5007450108041379 separated_ind 0.07942238267148015 0.27088627741969934 female_ind 0.555956678700361 0.4977582771429234 ck_acct_ind 0.7003610108303249 0.458928982771801 sv_acct_ind 0.5631768953068592 0.49689034574654706 sv_avg_bal 303.966184415435 679.0307677517072 ck_avg_bal 1204.600106206975 1983.0828939335584 cc_acct_ind 0.6425992779783394 0.48010162163955045 6 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 39.839285714285715 19.676267925613317 tot_income 27899.803571428572 29764.716274273254 tot_cust_years 3.5535714285714284 2.675951099275599 tot_children 0.8928571428571429 1.038855514121564 single_ind 0.35714285714285715 0.4834937784152282 married_ind 0.44642857142857145 0.5016207497934835 separated_ind 0.08928571428571429 0.28773635118317564 female_ind 0.5178571428571429 0.5042031130613726 ck_acct_ind 0.7321428571428571 0.4468504500113261 sv_acct_ind 0.5178571428571429 0.5042031130613726 sv_avg_bal 588.5857562765214 1538.3772829943812 ck_avg_bal 2133.470746955862 4196.524665266171 cc_acct_ind 0.625 0.48850421045919723 7 NOS_USR VAL_ADS2 12 cc_acct_ind 1 none 0 false IncludeConstant tot_age 44.467289719626166 19.603772142312902 tot_income 23650.906542056076 20500.314193879738 tot_cust_years 3.878504672897196 2.539165961617778 tot_children 0.6448598130841121 1.0302256251194661 single_ind 0.34579439252336447 0.47786482138302394 married_ind 0.5233644859813084 0.5018041

---

## Scoring da Regressão Logística

### Objetivo

Depois de construir um modelo preditivo usando a função de Regressão Logística, o modelo pode ser passado para uma função de Scoring de Regressão Logística para a criação de uma tabela de scoring contendo os valores previstos da variável dependente. Isso é feito lendo o banco de dados de saída e o nome da tabela de saída criados pela função de regressão logística, referindo-se a eles aqui como banco de dados de modelo e nome de tabela de modelo, respectivamente.

Além de uma tabela de pontuação, esta função pode opcionalmente produzir os seguintes resultados.

- Succes Table
- Multi-Threshold Success Table
- Lift Table



---

1. Primeiro, vamos executar o scoring do modelo singular de regressão logística criado acima. Vamos também criar uma Lift Table e uma Sucess Table, preenchidas em uma tabela com _score_txt  ao fim do nome da tabela de modelo.

In [42]:
call ${XSPDB}.td_analyze('logisticscore',
                         'database=${XSPDB};
                          tablename=VAL_ADS;
                          modeldatabase=${QLID};
                          modeltablename=LogisticOut1;
                          outputdatabase=${QLID};
                          outputtablename=LogisticScore1;
                          estimate=Estimate;
                          probability=Probability;
                          retain=cc_acct_ind;
                          samplescoresize=10;
                          lifttable=true;
                          successtable=true;  
                          scoringmethod=scoreandevaluate');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,cust_id,cc_acct_ind,Probability,Estimate
1,1362765,1,0.7134597180915243,1
2,1362862,0,0.1618428270762774,0
3,1363232,1,0.40098090847502604,0
4,1363251,0,0.44822035268809973,0
5,1362852,1,0.6283042638327425,1
6,1362596,0,0.27013310963487935,0
7,1363333,0,0.8160701533675125,1
8,1363005,0,0.12947945866983807,0
9,1362526,1,0.8064044536195069,1
10,1363178,0,0.7105647899149466,1


In [43]:
SELECT * FROM ${QLID}.LogisticScore1_txt;

partId XmlModel 1 1 <?xml version="1.0" encoding="UTF-8" standalone="no"?> Threshold Prob___Case 11________Case 10__________Case 01__________Case 00 0.00____________468_____________0_______________279_____________0 0.05____________468_____________0_______________278_____________1 0.10____________468_____________0_______________273_____________6 0.15____________468_____________0_______________268_____________11 0.20____________466_____________2_______________251_____________28 0.25____________460_____________8_______________233_____________46 0.30____________454_____________14______________209_____________70 0.35____________441_____________27______________184_____________95 0.40____________432_____________36______________158_____________121 0.45____________418_____________50______________142_____________137 0.50____________405_____________63______________131_____________148 0.55____________385_____________83______________115_____________164 0.60____________356_____________112_____________93______________186 0.65____________315_____________153_____________79______________200 0.70____________282_____________186_____________66______________213 0.75____________235_____________233_____________50______________229 0.80____________176_____________292_____________36______________243 0.85____________105_____________363_____________20______________259 0.90____________48______________420_____________6_______________273 0.95____________5_______________463_____________2_______________277 Decile_____Count_____Response__ResP_____CapResP___Lift_______CumResp__CumResP__CumCapResP CumLift 1_________74.00_____67.00_____90.54_____14.32_____1.45______67.00_____90.54_____14.32_____1.45 2_________75.00_____60.00_____80.00_____12.82_____1.28______127.00____85.23_____27.14_____1.36 3_________75.00_____61.00_____81.33_____13.03_____1.30______188.00____83.93_____40.17_____1.34 4_________74.00_____59.00_____79.73_____12.61_____1.27______247.00____82.89_____52.78_____1.32 5_________75.00_____53.00_____70.67_____11.32_____1.13______300.00____80.43_____64.10_____1.28 6_________75.00_____55.00_____73.33_____11.75_____1.17______355.00____79.24_____75.85_____1.26 7_________74.00_____42.00_____56.76_____8.97______0.91______397.00____76.05_____84.83_____1.21 8_________75.00_____36.00_____48.00_____7.69______0.77______433.00____72.53_____92.52_____1.16 9_________75.00_____24.00_____32.00_____5.13______0.51______457.00____68.01_____97.65_____1.09 10________75.00_____11.00_____14.67_____2.35______0.23______468.00____62.65_____100.00____1.00

In [44]:
SELECT XMLSERIALIZE(Content X.Dot) as XMLText
FROM (SELECT * FROM ${QLID}.LogisticScore1_txt) AS C,
XMLTable (
'//*'
PASSING CREATEXML(C.XmlModel)
) AS X ("Dot")
;

XMLText 1 Threshold Prob___Case 11________Case 10__________Case 01__________Case 00 0.00____________468_____________0_______________279_____________0 0.05____________468_____________0_______________278_____________1 0.10____________468_____________0_______________273_____________6 0.15____________468_____________0_______________268_____________11 0.20____________466_____________2_______________251_____________28 0.25____________460_____________8_______________233_____________46 0.30____________454_____________14______________209_____________70 0.35____________441_____________27______________184_____________95 0.40____________432_____________36______________158_____________121 0.45____________418_____________50______________142_____________137 0.50____________405_____________63______________131_____________148 0.55____________385_____________83______________115_____________164 0.60____________356_____________112_____________93______________186 0.65____________315_____________153_____________79______________200 0.70____________282_____________186_____________66______________213 0.75____________235_____________233_____________50______________229 0.80____________176_____________292_____________36______________243 0.85____________105_____________363_____________20______________259 0.90____________48______________420_____________6_______________273 0.95____________5_______________463_____________2_______________277 Decile_____Count_____Response__ResP_____CapResP___Lift_______CumResp__CumResP__CumCapResP CumLift 1_________74.00_____67.00_____90.54_____14.32_____1.45______67.00_____90.54_____14.32_____1.45 2_________75.00_____60.00_____80.00_____12.82_____1.28______127.00____85.23_____27.14_____1.36 3_________75.00_____61.00_____81.33_____13.03_____1.30______188.00____83.93_____40.17_____1.34 4_________74.00_____59.00_____79.73_____12.61_____1.27______247.00____82.89_____52.78_____1.32 5_________75.00_____53.00_____70.67_____11.32_____1.13______300.00____80.43_____64.10_____1.28 6_________75.00_____55.00_____73.33_____11.75_____1.17______355.00____79.24_____75.85_____1.26 7_________74.00_____42.00_____56.76_____8.97______0.91______397.00____76.05_____84.83_____1.21 8_________75.00_____36.00_____48.00_____7.69______0.77______433.00____72.53_____92.52_____1.16 9_________75.00_____24.00_____32.00_____5.13______0.51______457.00____68.01_____97.65_____1.09 10________75.00_____11.00_____14.67_____2.35______0.23______468.00____62.65_____100.00____1.00 2 Threshold Prob___Case 11________Case 10__________Case 01__________Case 00 0.00____________468_____________0_______________279_____________0 0.05____________468_____________0_______________278_____________1 0.10____________468_____________0_______________273_____________6 0.15____________468_____________0_______________268_____________11 0.20____________466_____________2_______________251_____________28 0.25____________460_____________8_______________233_____________46 0.30____________454_____________14______________209_____________70 0.35____________441_____________27______________184_____________95 0.40____________432_____________36______________158_____________121 0.45____________418_____________50______________142_____________137 0.50____________405_____________63______________131_____________148 0.55____________385_____________83______________115_____________164 0.60____________356_____________112_____________93______________186 0.65____________315_____________153_____________79______________200 0.70____________282_____________186_____________66______________213 0.75____________235_____________233_____________50______________229 0.80____________176_____________292_____________36______________243 0.85____________105_____________363_____________20______________259 0.90____________48______________420_____________6_______________273 0.95____________5_______________463_____________2_______________277 Decile_____Count_____Response__ResP_____CapResP___Lift_______CumResp__CumResP__CumCapResP CumL

2. Em seguida, vamos executar o scoring do modelo de regressão logística criado acima para cada estado.

---

In [45]:
call ${XSPDB}.td_analyze('logisticscore',
                         'database=${XSPDB};
                          tablename=VAL_ADS2;
                          modeldatabase=${QLID};
                          modeltablename=LogisticOut2;
                          outputdatabase=${QLID};
                          outputtablename=LogisticScore2;
                          estimate=Estimate;
                          probability=Probability;
                          retain=cc_acct_ind;
                          samplescoresize=10;
                          scoringmethod=score');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,cust_id,state_code,cc_acct_ind,Probability,Estimate
1,1362677,OTHER,0,0.7532874523610942,1
2,1362883,IL,1,0.39178240100620165,0
3,1362728,NY,0,0.5541210015890233,1
4,1363006,TX,1,0.687730236608376,1
5,1363017,CA,1,0.5811144546908545,1
6,1362960,OTHER,1,0.27557520692375326,0
7,1362602,CA,1,0.9161636297952092,1
8,1362909,IL,1,0.9999999999979262,1
9,1362862,CA,0,0.16574576878800046,0
10,1362836,OTHER,1,0.9088645138147252,1


### Power BI
Para finalizar este notebook, vamos visualizar o resultado da regressão logística através do PowerBI para identificar qual variável possui maior influência!

---